In [1]:
# import libraries
import cv2
import numpy as np
import glob
import os
from itertools import combinations

# Function to detect keypoints and compute descriptors
def detect_and_compute(detector, descriptor, img):
    keypoints = detector.detect(img, None)
    keypoints, descriptors = descriptor.compute(img, keypoints)
    return keypoints, descriptors

# Function to read images from a directory
def read_images(train_images_dir):
    train_images = [cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in glob.glob(os.path.join(train_images_dir, "*"))]
    return train_images

# Function to match descriptors and return the matches
def match_descriptors(descriptor_extractor, query_descriptors, train_descriptors):
    # Match descriptors.
    matches = descriptor_extractor.knnMatch(query_descriptors, train_descriptors, k=2)
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    return good_matches

def main(train_images_dir):
    # Create SIFT detector and descriptor
    sift = cv2.SIFT_create()

    # Read images
    images = read_images(train_images_dir)

    # Detect keypoints and compute descriptors for all images
    keypoints_descriptors = [detect_and_compute(sift, sift, img) for img in images]

    # Initialize a list to keep track of which keypoints appear in all combinations
    matches_in_all_combinations = set(range(len(keypoints_descriptors[0][0])))

    # Initialize a list to store the matches for each combination
    all_matches = []

    # Initialize a list to store the combinations of images
    combinations_of_images = list(combinations(range(len(images)), 2))

    # For each combination of images
    for combination in combinations_of_images:
        # Initialize chains with matches between the first and second image of the combination
        matches = match_descriptors(cv2.BFMatcher(cv2.NORM_L2), keypoints_descriptors[combination[0]][1], keypoints_descriptors[combination[1]][1])
        chains = [[m.queryIdx, m.trainIdx] for m in matches]

        # Update the set of keypoints that appear in all combinations
        matches_in_all_combinations &= set(chain[0] for chain in chains)

        # Add the chains to the list of all matches
        all_matches.append(chains)

    # Filter the matches to include only those that appear in all combinations
    all_matches = [[chain for chain in chains if chain[0] in matches_in_all_combinations] for chains in all_matches]

    # Prepare the result image
    h, w = max(img.shape[:2] for img in images)
    output_img = np.zeros((2 * h, 2 * w, 3), dtype="uint8")

    # Place each image at its own location in a 2x2 grid
    output_img[:h, :w, :] = cv2.cvtColor(images[0], cv2.COLOR_GRAY2BGR)
    output_img[:h, w:, :] = cv2.cvtColor(images[1], cv2.COLOR_GRAY2BGR)
    output_img[h:, :w, :] = cv2.cvtColor(images[2], cv2.COLOR_GRAY2BGR)
    output_img[h:, w:, :] = cv2.cvtColor(images[3], cv2.COLOR_GRAY2BGR)

    # List of colors (red, green, blue, yellow, pink, and cyan)
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

    # Draw the matches for each combination with a unique color
    for i, chains in enumerate(all_matches):
        for chain in chains:
            pt1 = tuple(map(int, keypoints_descriptors[combinations_of_images[i][0]][0][chain[0]].pt))
            pt2 = tuple(map(int, keypoints_descriptors[combinations_of_images[i][1]][0][chain[1]].pt))
            pt1 = ((pt1[0] + combinations_of_images[i][0] % 2 * w, pt1[1] + combinations_of_images[i][0] // 2 * h))
            pt2 = ((pt2[0] + combinations_of_images[i][1] % 2 * w, pt2[1] + combinations_of_images[i][1] // 2 * h))
            cv2.line(output_img, pt1, pt2, colors[i], 2)

    # Save result image
    cv2.imwrite('result.png', output_img)

    # Print the number of matches for each color
    for i, chains in enumerate(all_matches):
        print(f"Number of matches in color {colors[i]}: {len(chains)}")


# Usage
main('all/')


Number of matches in color (255, 0, 0): 4
Number of matches in color (0, 255, 0): 4
Number of matches in color (0, 0, 255): 4
Number of matches in color (255, 255, 0): 4
Number of matches in color (255, 0, 255): 4
Number of matches in color (0, 255, 255): 4
